In [1]:
#%% Importar librerias
import pyodbc
import pandas as pd

In [2]:
#%%RUTA DEL ARCHIVO
ruta = r"C:\Users\bgacitua\Desktop\Reportes Corrección de Marcas\Reportes_marcas_rflex\reporte_turnos_sin_marcas_20250819081940.xlsx"

In [3]:
#%% Configuración de conexión
SERVER = '192.9.200.84' 
DATABASE = 'MorphoManager'  
USERNAME = 'rrhh_morpho'  
PASSWORD = 'PQ90@ZT*'  

#%% Cadena de conexión para SQL Server
try:
    connection_string = (
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={SERVER};'
        f'DATABASE={DATABASE};'
        f'UID={USERNAME};'
        f'PWD={PASSWORD};'
        'TrustServerCertificate=yes;'  # Para evitar problemas con certificados SSL
    )
    
    cnxn = pyodbc.connect(connection_string)
    print("¡Conexión a SQL Server establecida exitosamente! Lista para múltiples consultas.")

except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"Error al conectar a SQL Server: {ex}")
    if sqlstate == '28000':
        print("Causa probable: Error de autenticación (usuario o contraseña incorrectos).")
    elif sqlstate == '08001':
        print("Causa probable: El servidor SQL Server no está accesible, el nombre del servidor es incorrecto, o el puerto (1433 por defecto) está bloqueado por un firewall.")
    else:
        print("Consulta el mensaje de error para más detalles.")


¡Conexión a SQL Server establecida exitosamente! Lista para múltiples consultas.


In [4]:
# %%
import datetime
fecha_actual_py = datetime.datetime.now()
fecha_inicio_sql = fecha_actual_py.replace(day=1).strftime('%Y-%m-%d')  # Primer día del mes actual
fecha_fin_sql = fecha_actual_py.strftime('%Y-%m-%d')  # Fecha actual

print(f"Fecha inicio: {fecha_inicio_sql}")
print(f"Fecha fin: {fecha_fin_sql}")
#%%   
# --- 3. Bloque para Consultas Iterativas y Transformación a DataFrame ---
# Puedes cambiar la consulta SQL aquí cada vez que quieras probar una nueva.
sql_query = f"""
-- Versión optimizada con rango de fechas automático
-- Las fechas se calculan automáticamente desde Python
DECLARE @fecha_inicio DATE = '{fecha_inicio_sql}';
DECLARE @fecha_fin DATE = '{fecha_fin_sql}';

WITH fechas_rango AS (
    -- Generar secuencia de fechas entre inicio y fin
    SELECT @fecha_inicio AS [FECHA_SANTIAGO]
    UNION ALL
    SELECT DATEADD(DAY, 1, [FECHA_SANTIAGO])
    FROM fechas_rango
    WHERE DATEADD(DAY, 1, [FECHA_SANTIAGO]) <= @fecha_fin
),
todos_trabajadores AS (
    -- Obtener todos los trabajadores activos
    SELECT DISTINCT 
        u.[ID] AS [USER_ID],
        u.[FIRSTNAME],
        u.[LASTNAME],
        u.[EMPLOYEEID]
    FROM [MorphoManager].[dbo].[User_] AS u
    WHERE u.[EMPLOYEEID] IS NOT NULL 
    AND u.[EMPLOYEEID] != ''  -- Filtrar empleados válidos
),
combinaciones_trabajador_fecha AS (
    -- Crear matriz completa trabajador x fecha
    SELECT 
        t.[USER_ID],
        t.[FIRSTNAME],
        t.[LASTNAME], 
        t.[EMPLOYEEID],
        f.[FECHA_SANTIAGO]
    FROM todos_trabajadores t
    CROSS JOIN fechas_rango f
)
SELECT 
    -- Información del trabajador
    ctf.[FIRSTNAME],
    ctf.[LASTNAME],
    ctf.[EMPLOYEEID],
    ctf.[FECHA_SANTIAGO],
    
    -- Conteo de registros por tipo (0 si no hay marcas)
    COALESCE(SUM(CASE WHEN m.[FUNCTIONKEYTEXT] = 'IN' THEN 1 ELSE 0 END), 0) AS [KEY_IN],
    COALESCE(SUM(CASE WHEN m.[FUNCTIONKEYTEXT] = 'OUT' THEN 1 ELSE 0 END), 0) AS [KEY_OUT],
    COALESCE(SUM(CASE WHEN m.[FUNCTIONKEYTEXT] = 'No Key' THEN 1 ELSE 0 END), 0) AS [NO_KEY],
    
    -- Estado de asistencia
    CASE 
        WHEN COUNT(m.[ID]) = 0 THEN 'Ausente'
        WHEN COUNT(m.[ID]) >= 1 THEN 'Asiste'
    END AS [ESTADO_ASISTENCIA],
    
    -- Información adicional
    COALESCE(COUNT(DISTINCT bd.[NAME_]), 0) AS [CANTIDAD_DISPOSITIVOS],
    COALESCE(COUNT(m.[ID]), 0) AS [TOTAL_REGISTROS],
    
    -- Información de horarios (opcional)
    COALESCE(MIN(CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS TIME)), '00:00:00') AS [PRIMERA_MARCA],
    COALESCE(MAX(CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS TIME)), '00:00:00') AS [ULTIMA_MARCA]
    
FROM combinaciones_trabajador_fecha ctf
LEFT JOIN [MorphoManager].[dbo].[AccessLog] AS m 
    ON ctf.[USER_ID] = m.[USERID]
    AND ctf.[FECHA_SANTIAGO] = CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS DATE)
    AND CAST(m.[LOGDATETIME] AT TIME ZONE 'UTC' AT TIME ZONE 'Pacific SA Standard Time' AS DATE) BETWEEN @fecha_inicio AND @fecha_fin
LEFT JOIN [MorphoManager].[dbo].[BiometricDevice] AS bd 
    ON m.[MORPHOACCESSID] = bd.[ID]
    
GROUP BY 
    ctf.[FIRSTNAME],
    ctf.[LASTNAME],
    ctf.[EMPLOYEEID],
    ctf.[FECHA_SANTIAGO],
    ctf.[USER_ID]
    
ORDER BY 
    ctf.[LASTNAME], ctf.[FIRSTNAME], ctf.[FECHA_SANTIAGO] DESC
OPTION (MAXRECURSION 366);  -- Permite hasta 366 días en la recursión
"""

Fecha inicio: 2025-08-01
Fecha fin: 2025-08-19


In [5]:
#%% TRANSFORMAMOS CONSULTA EN DATAFRAME PARA TRABAJAR CON PANDAS

try:
    # Utiliza pandas.read_sql para ejecutar la consulta con la conexión ya abierta
    df_resultado = pd.read_sql(sql_query, cnxn)

    print("\n--- ¡Consulta ejecutada y DataFrame creado! ---")
    #print(f"Consulta ejecutada:\n{sql_query.strip()}") # strip() para limpiar espacios al inicio/fin

    print("\nPrimeras 5 filas del DataFrame:")
    print(df_resultado.head())

    print(f"\nNúmero de filas y columnas: {df_resultado.shape}")
    print("Columnas y tipos de datos:")
    df_resultado.info() # df.info() no retorna, solo imprime, por eso no se usa print()

except pyodbc.Error as ex:
    # Manejo de errores de consulta (ej. tabla no existe, sintaxis SQL incorrecta)
    print(f"Error al ejecutar la consulta SQL: {ex}")
    print("Por favor, revisa tu consulta SQL o si la tabla/columna existe y tienes permisos.")
except Exception as e:
    # Manejo de cualquier otro error inesperado
    print(f"Ocurrió un error inesperado durante el procesamiento: {e}")

# NOTA: La conexión NO se cierra aquí. Se mantiene abierta para la siguiente consulta.

#%% SEPARAR DV DEL RUT
# Paso 1: Identificar las filas que contienen el delimitador "-"
tiene_delimitador = df_resultado['EMPLOYEEID'].astype(str).str.contains('-')

# Paso 2: Crear la nueva columna con los valores después del delimitador
df_resultado['ID_ADICIONAL'] = df_resultado['EMPLOYEEID'].astype(str).str.split('-').str[1].fillna('')

# Paso 3: Actualizar la columna EMPLOYEEID para mantener solo la parte antes del delimitador
df_resultado.loc[tiene_delimitador, 'EMPLOYEEID'] = df_resultado.loc[tiene_delimitador, 'EMPLOYEEID'].astype(str).str.split('-').str[0]

try:
    df_resultado['EMPLOYEEID'] = pd.to_numeric(df_resultado['EMPLOYEEID'], errors='coerce')
    # errors='coerce' convertirá los valores que no puedan ser convertidos a números en NaN
    print("\n'EMPLOYEEID' se convirtió a tipo numérico (NaN si no pudo convertir).")
except Exception as e:
    print(f"\nAdvertencia: No se pudo convertir 'EMPLOYEEID' a numérico debido a: {e}")

# Mostrar el DataFrame después de la transformación
print("\nDataFrame después de la transformación:")
print(df_resultado[['EMPLOYEEID', 'ID_ADICIONAL']].head(5))

df_resultado.info()
#%%CREAR COLUMNA DE BUSQUEDA RUT + FECHA

df_resultado['EMPLOYEEID_STR'] = df_resultado['EMPLOYEEID'].astype(str) 
df_resultado['EMPLOYEEID_STR'] = df_resultado['EMPLOYEEID_STR'].str.replace(r'\.0$', '', regex=True)
#df_resultado['EMPLOYEEID'] = df_resultado['EMPLOYEEID'].astype(int) 
df_resultado['FECHA_SANTIAGO_STR'] = df_resultado['FECHA_SANTIAGO'].astype(str)
df_resultado.info()
df_resultado['BusquedaRUT'] = df_resultado['EMPLOYEEID_STR'] + ";" + df_resultado['FECHA_SANTIAGO_STR']

print(df_resultado)

C:\Users\bgacitua\AppData\Local\Temp\ipykernel_18064\470403224.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultado = pd.read_sql(sql_query, cnxn)



--- ¡Consulta ejecutada y DataFrame creado! ---

Primeras 5 filas del DataFrame:
   FIRSTNAME LASTNAME  EMPLOYEEID FECHA_SANTIAGO  KEY_IN  KEY_OUT  NO_KEY  \
0  Yessenia   Abanto   25022430-3     2025-08-19       0        0       0   
1  Yessenia   Abanto   25022430-3     2025-08-18       0        0      17   
2  Yessenia   Abanto   25022430-3     2025-08-17       0        0       0   
3  Yessenia   Abanto   25022430-3     2025-08-16       0        0       7   
4  Yessenia   Abanto   25022430-3     2025-08-15       0        0       0   

  ESTADO_ASISTENCIA  CANTIDAD_DISPOSITIVOS  TOTAL_REGISTROS PRIMERA_MARCA  \
0           Ausente                      0                0      00:00:00   
1            Asiste                     10               17      11:35:29   
2           Ausente                      0                0      00:00:00   
3            Asiste                      5                7      07:15:29   
4           Ausente                      0                0      00:00

In [6]:
#%%Info del Dataframe SQL
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14364 entries, 0 to 14363
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   FIRSTNAME              14364 non-null  object 
 1   LASTNAME               14364 non-null  object 
 2   EMPLOYEEID             13870 non-null  float64
 3   FECHA_SANTIAGO         14364 non-null  object 
 4   KEY_IN                 14364 non-null  int64  
 5   KEY_OUT                14364 non-null  int64  
 6   NO_KEY                 14364 non-null  int64  
 7   ESTADO_ASISTENCIA      14364 non-null  object 
 8   CANTIDAD_DISPOSITIVOS  14364 non-null  int64  
 9   TOTAL_REGISTROS        14364 non-null  int64  
 10  PRIMERA_MARCA          14364 non-null  object 
 11  ULTIMA_MARCA           14364 non-null  object 
 12  ID_ADICIONAL           14364 non-null  object 
 13  EMPLOYEEID_STR         14364 non-null  object 
 14  FECHA_SANTIAGO_STR     14364 non-null  object 
 15  Bu

In [ ]:
#%% Exportar archivo CSV con timestamp automático
import datetime
import os

# Crear timestamp para archivo único
now = datetime.datetime.now()
timestamp_str = now.strftime("%Y%m%d_%H%M%S")
titulo = "Respaldo_Morpho_AccessLog"

# Crear carpeta si no existe
carpeta_destino = r"C:\Users\bgacitua\Desktop\Repositorio_GitHub\Scripts de Python\Respaldos_Biometrico"
os.makedirs(carpeta_destino, exist_ok=True)

# Nombre del archivo CSV
nombre_archivo = os.path.join(carpeta_destino, f"{titulo}_{timestamp_str}.csv")

try:
    # Exportar a CSV con encoding UTF-8
    df_resultado.to_csv(nombre_archivo, index=False, encoding='utf-8-sig', sep=';')
    
    print(f"\n✅ ¡Respaldo CSV generado exitosamente!")
    print(f"📁 Archivo: {nombre_archivo}")
    print(f"📊 Registros exportados: {len(df_resultado):,}")
    print(f"📋 Columnas incluidas: {len(df_resultado.columns)}")
    print(f"📅 Fecha del respaldo: {now.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"💾 Tamaño del archivo: {os.path.getsize(nombre_archivo) / 1024:.2f} KB")
    
    # Estadísticas del DataFrame
    print(f"\n📈 ESTADÍSTICAS DEL RESPALDO:")
    print(f"   Total empleados únicos: {df_resultado['EMPLOYEEID'].nunique()}")
    print(f"   Rango de fechas: {df_resultado['FECHA_SANTIAGO'].min()} a {df_resultado['FECHA_SANTIAGO'].max()}")
    print(f"   Empleados con asistencia: {(df_resultado['ESTADO_ASISTENCIA'] == 'Asiste').sum():,}")
    print(f"   Empleados ausentes: {(df_resultado['ESTADO_ASISTENCIA'] == 'Ausente').sum():,}")

except Exception as e:
    print(f"❌ Error al generar el respaldo: {str(e)}")


¡Reporte exportado exitosamente en formato Excel a 'C:/Users/bgacitua/Desktop/Reportes Corrección de Marcas/Planillas_Python_Spyder/df_rellenar_con_observaciones_2025-08-19_08-22-14.xlsx'!
Registros exportados: 66
Columnas incluidas: ['RUT', 'Nombre', 'Unidad', 'Puesto', 'Fecha', 'Tipo de turno', 'Entrada programada', 'Salida programada', 'Marca de entrada', 'Marca de salida', 'DV', 'Busqueda', 'KEY_IN', 'KEY_OUT', 'NO_KEY', 'PRIMERA_MARCA', 'ULTIMA_MARCA', 'Marca de entrada Generados', 'Marca de salida Generados', 'Observación de turno']
Registros con observación de turno: 16


In [15]:
# %%
# --- 4. Cerrar la Conexión (al finalizar todas las operaciones) ---
if 'cnxn' in locals() and cnxn: # Verificar si 'cnxn' existe y no es None
    try:
        cnxn.close()
        print("\nConexión a SQL Server cerrada exitosamente.")
    except Exception as e:
        print(f"Error al cerrar la conexión: {e}")
else:
    print("La conexión no fue establecida o ya está cerrada.")


Conexión a SQL Server cerrada exitosamente.
